In [151]:
# Julia 0.7
using CSV, DataFrames, Gadfly, Statistics, LinearAlgebra, GLM, CategoricalArrays

# Chargement du jeu de données d'entraînement

In [152]:
train = CSV.read("train.csv")

head(train)

,ID,Name,Platform,Year_of_Release,Genre,Publisher,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,NA_Sales,Global_Sales
,Int64⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰
1,1,Zoo Keeper,DS,2004,Puzzle,Ignition Entertainment,0.0505242,0.0100241,74,40,7.5,13,Buddiez- Inc.,E,0.101432,0.172605
2,2,Do-Konjou Shougakussei: Bon Bita - Hadaka no Choujou Ketsusen!! Bita vs. Dokuro Dei!,DS,2010,Action,Namco Bandai Games,0.0392108,0.00179053,missing,missing,missing,missing,missing,missing,0.000439844,0.0432509
3,3,Power Play Pool,DS,2006,Sports,System 3 Arcade Software,0.000374026,0.00179738,missing,missing,missing,missing,System 3,E,0.0308291,0.0412706
4,4,SpongeBob SquarePants: Revenge of the Flying Dutchman,PS2,2002,Platform,THQ,0.00138205,0.14001,missing,missing,6.6,41,Big Sky Software,E,0.52833,1.08082
5,5,Deception IV: Blood Ties,PSV,2014,Action,Tecmo Koei,0.0694222,0.00953506,67,19,7.5,61,Koei Tecmo Games,M,0.0206288,0.131074
6,6,Binary Domain,PS3,2012,Action,Sega,0.14039,0.0302705,72,33,8.2,179,Sega,M,0.0890602,0.331157


# Analyse exploratoire

In [153]:
NA_Sales = collect(train[:,:NA_Sales])
Global_Sales = collect(train[:,:Global_Sales])
Critic_Score = collect(train[:,:Critic_Score])
Critic_Count = collect(train[:,:Critic_Count])
User_Score = collect(train[:,:User_Score])
User_Count = collect(train[:,:User_Count])
Other_Sales = collect(train[:,:Other_Sales])
JP_Sales = collect(train[:,:JP_Sales])

platformMissing = collect(train[:Platform])
yearMissing = collect(train[:Year_of_Release])
ratingMissing = collect(train[:Rating])
genreMissing = collect(train[:Genre])

for i in eachindex(platformMissing) 
    if ismissing(platformMissing[i])
        platformMissing[i] = "NoPlatform"
    end
end

platform = categorical(platformMissing)

for i in eachindex(yearMissing) 
    if ismissing(yearMissing[i])
        yearMissing[i] = 1000
    end
end
year = categorical(yearMissing)

for i in eachindex(ratingMissing) 
    if ismissing(ratingMissing[i])
        ratingMissing[i] = "missingRating"
    end
end
rating = categorical(ratingMissing)

## GENRE

for i in eachindex(genreMissing) 
    if ismissing(genreMissing[i])
        genreMissing[i] = "missing"
    end
end

genre = categorical(genreMissing)

moyenneCriticScore = floor(mean(collect(skipmissing(train[:,:Critic_Score]))))
moyenneCriticCount =floor(mean(collect(skipmissing(train[:,:Critic_Count]))))
moyenneUser_Score = floor(mean(collect(skipmissing(train[:,:User_Score]))))
moyenneUser_Count = floor(mean(collect(skipmissing(train[:,:User_Count]))))


for i in eachindex(Critic_Score) 
    if ismissing(Critic_Score[i]) || ismissing(Critic_Count[i]) || ismissing(User_Score[i]) || ismissing(User_Count[i])
        Critic_Score[i]= moyenneCriticScore
        Critic_Count[i]= moyenneCriticCount
        User_Score[i]= moyenneUser_Score
        User_Count[i]= moyenneUser_Count
    end
end


df1 = DataFrame(Global = Global_Sales,JP = JP_Sales, Platform = platform, Year = year, Rating = rating,
    Other = Other_Sales, CC=Critic_Count, US=User_Score,UC=User_Count, Genre = genre)

df2 = DataFrame(NA = NA_Sales,JP = JP_Sales, Platform = platform, Year = year, Rating = rating,
    Other = Other_Sales, CC=Critic_Count, US=User_Score,UC=User_Count, Genre = genre)


# M₁=lm(@formula(Global ~  JP + Platform + Year + Rating + Other + CC + US + UC), df1)
# M₂=lm(@formula(NA ~ JP + Platform + Year + Rating + Other + CC + US + UC), df2)

M₁=lm(@formula(Global ~  JP + Platform + Rating + Year + Other + CC + US + UC + Genre), df1)
M₂=lm(@formula(NA ~ JP + Platform + Rating + Year + Other +CC + US + UC + Genre ), df2)

println(deviance(M₁))
println(deviance(M₂))
println(adjr2(M₁))
println(adjr2(M₂))


8853.776506099122
4714.534067785332
0.7339404183082994
0.5141224164828972


# Charger les données de l'échantillon de test

In [154]:
test = CSV.read("test.csv")
head(test)

,ID,Platform,Year_of_Release,Genre,Publisher,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
,Int64⍰,String⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,String⍰,String⍰
1,14095,GBA,2002,Fighting,Acclaim Entertainment,0.000520983,0.00025736,24,11,3.7,10,Powerhead Games,E
2,14096,PS3,2008,Action,Codemasters,0.000966418,0.0204632,72,35,7.7,27,4J Studios,T
3,14097,N64,1997,Sports,Konami Digital Entertainment,0.259834,0.0489383,missing,missing,missing,missing,missing,missing
4,14098,GC,2002,Fighting,Midway Games,0.00088793,0.00826891,81,20,8.7,43,Midway,M
5,14099,X360,2010,Role-Playing,Sega,0.000439246,0.0388025,63,68,7.2,201,Obsidian Entertainment,M
6,14100,DS,2009,Adventure,Ubisoft,0.001625,0.00016853,missing,missing,7.5,4,Longtail Studios,T


In [155]:
n = size(test,1)

Critic_ScoreTest = collect(test[:,:Critic_Score])
Critic_CountTest = collect(test[:,:Critic_Count])
User_ScoreTest = collect(test[:,:User_Score])
User_CountTest = collect(test[:,:User_Count])
Other_SalesTest = collect(test[:,:Other_Sales])
JP_SalesTest = collect(test[:,:JP_Sales])

for i in eachindex(Critic_ScoreTest) 
    if ismissing(Critic_ScoreTest[i]) || ismissing(Critic_CountTest[i]) || ismissing(User_ScoreTest[i]) || ismissing(User_CountTest[i])
        Critic_ScoreTest[i]= moyenneCriticScore
        Critic_CountTest[i]= moyenneCriticCount
        User_ScoreTest[i]= moyenneUser_Score
        User_CountTest[i]= moyenneUser_Count
    end
end

platformNonCategorical = collect(test[:Platform])


for i in eachindex(platformNonCategorical)
    if(platformNonCategorical[i] == "GG")
        platformNonCategorical[i] = "PC"
    end
end

platformTest = categorical(platformNonCategorical)

yearNonCategorical = collect(test[:Year_of_Release])

# On eleve les donnes manquantes
for i in eachindex(yearNonCategorical)
    if(ismissing(yearNonCategorical[i]))
        yearNonCategorical[i] = 1000
    else
        if yearNonCategorical[i] == 2020
            yearNonCategorical[i] = 1996
        elseif yearNonCategorical[i] == 1994
            yearNonCategorical[i] = 1996
        end
    end
end


yearTest = categorical(yearNonCategorical)

ratingNonCategorical = collect(test[:Rating])

for i in eachindex(ratingNonCategorical)
    if(ismissing(ratingNonCategorical[i]))
        ratingNonCategorical[i] = "missingRating"
    end
end

ratingTest = categorical(ratingNonCategorical)

##Genre

genreNonCategorical = collect(test[:,:Genre])

for i in eachindex(genreNonCategorical)
    if(ismissing(genreNonCategorical[i]))
        genreNonCategorical[i] = "missing"
    end
end

genreTest = categorical(genreNonCategorical)
# dfTest = DataFrame(one = ones(n),JP = JP_SalesTest,Platform = platformTest, Year = yearTest, Rating = ratingTest, Other = Other_SalesTest, CC=Critic_CountTest, US=User_ScoreTest, UC=User_CountTest )

dfTest = DataFrame(one = ones(n),JP = JP_SalesTest,Platform = platformTest, Rating = ratingTest, Year = yearTest, Other = Other_SalesTest, CC=Critic_CountTest, US=User_ScoreTest, UC=User_CountTest, Genre = genreTest )

NA_SalesTest = predict(M₂,dfTest)

Global_SalesTest = predict(M₁,dfTest)

predictions = DataFrame(ID = test[:ID], NA_Sales = NA_SalesTest, Global_Sales = Global_SalesTest)
CSV.write("prediction_JP_Other_Platform_Year_Rating_Critic_User.csv",predictions)

"prediction_JP_Other_Platform_Year_Rating_Critic_User.csv"

In [70]:
whos(GLM)

| name                   |        size | summary                                    |
|:---------------------- | -----------:|:------------------------------------------ |
| @formula               |     0 bytes | getfield(StatsModels, Symbol("#@formula")) |
| AbstractContrasts      |   172 bytes | DataType                                   |
| Bernoulli              |    40 bytes | UnionAll                                   |
| Binomial               |    40 bytes | UnionAll                                   |
| CauchitLink            |   172 bytes | DataType                                   |
| CloglogLink            |   172 bytes | DataType                                   |
| ContrastsCoding        |   300 bytes | DataType                                   |
| DensePred              |   172 bytes | DataType                                   |
| DensePredChol          |    96 bytes | UnionAll                                   |
| DensePredQR            |    56 bytes | UnionAll                                   |
| DummyCoding            |   252 bytes | DataType                                   |
| EffectsCoding          |   252 bytes | DataType                                   |
| Formula                |   276 bytes | DataType                                   |
| GLM                    | 236.807 KiB | Module                                     |
| Gamma                  |    40 bytes | UnionAll                                   |
| GeneralizedLinearModel |   280 bytes | UnionAll                                   |
| GlmResp                |   200 bytes | UnionAll                                   |
| HelmertCoding          |   252 bytes | DataType                                   |
| IdentityLink           |   172 bytes | DataType                                   |
| InverseGaussian        |    40 bytes | UnionAll                                   |
| InverseLink            |   172 bytes | DataType                                   |
| InverseSquareLink      |   172 bytes | DataType                                   |
| LinPred                |   172 bytes | DataType                                   |
| LinPredModel           |   172 bytes | DataType                                   |
| LinearModel            |   160 bytes | UnionAll                                   |
| Link                   |   172 bytes | DataType                                   |
| LmResp                 |    80 bytes | UnionAll                                   |
| LogLink                |   172 bytes | DataType                                   |
| LogitLink              |   172 bytes | DataType                                   |
| ModelFrame             |   252 bytes | DataType                                   |
| ModelMatrix            |    80 bytes | UnionAll                                   |
| NegativeBinomial       |    40 bytes | UnionAll                                   |
| NegativeBinomialLink   |   188 bytes | DataType                                   |
| Normal                 |    40 bytes | UnionAll                                   |
| Poisson                |    40 bytes | UnionAll                                   |
| ProbitLink             |   172 bytes | DataType                                   |
| SqrtLink               |   172 bytes | DataType                                   |
| StatsModels            | 241.042 KiB | Module                                     |
| adjr2                  |     0 bytes | typeof(adjr2)                              |
| adjr²                  |     0 bytes | typeof(adjr2)                              |
| canonicallink          |     0 bytes | typeof(canonicallink)                      |
| coef                   |     0 bytes | typeof(coef)                               |
| coefnames              |     0 bytes | typeof(coefnames)                          |
| coeftable              |     0 bytes | typeof(coeftable)                          |
| confint                |     0 bytes | typeof(confint)                            |
| delbeta!               |     0 bytes | typeof(delbeta!)                           |
| deviance               |     0 bytes | typeof(deviance)                           |
| devresid               |     0 bytes | typeof(devresid)                           |
| dof                    |     0 bytes | typeof(dof)                                |
| dof_residual           |     0 bytes | typeof(dof_residual)                       |
| dropterm               |     0 bytes | typeof(dropterm)                           |
| fit                    |     0 bytes | typeof(fit)                                |
| fit!                   |     0 bytes | typeof(fit!)                               |
| formula                |     0 bytes | typeof(formula)                            |
| ftest                  |     0 bytes | typeof(ftest)                              |
| glm                    |     0 bytes | typeof(glm)                                |
| glmvar                 |     0 bytes | typeof(glmvar)                             |
| inverselink            |     0 bytes | typeof(inverselink)                        |
| linkfun                |     0 bytes | typeof(linkfun)                            |
| linkinv                |     0 bytes | typeof(linkinv)                            |
| linpred                |     0 bytes | typeof(linpred)                            |
| linpred!               |     0 bytes | typeof(linpred!)                           |
| lm                     |     0 bytes | typeof(lm)                                 |
| logistic               |     0 bytes | typeof(logistic)                           |
| logit                  |     0 bytes | typeof(logit)                              |
| loglikelihood          |     0 bytes | typeof(loglikelihood)                      |
| model_response         |     0 bytes | typeof(model_response)                     |
| mueta                  |     0 bytes | typeof(mueta)                              |
| mustart                |     0 bytes | typeof(mustart)                            |
| negbin                 |     0 bytes | typeof(negbin)                             |
| nobs                   |     0 bytes | typeof(nobs)                               |
| nulldeviance           |     0 bytes | typeof(nulldeviance)                       |
| nullloglikelihood      |     0 bytes | typeof(nullloglikelihood)                  |
| predict                |     0 bytes | typeof(predict)                            |
| r2                     |     0 bytes | typeof(r2)                                 |
| residuals              |     0 bytes | typeof(residuals)                          |
| r²                     |     0 bytes | typeof(r2)                                 |
| setcontrasts!          |     0 bytes | typeof(setcontrasts!)                      |
| stderror               |     0 bytes | typeof(stderror)                           |
| updateμ!               |     0 bytes | typeof(updateμ!)                           |
| vcov                   |     0 bytes | typeof(vcov)                               |
| wrkresp                |     0 bytes | typeof(wrkresp)                            |
